# RAG Medical Research
### with Groq


In [1]:
# Import necessary libraries
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint
import warnings
from langchain_groq import ChatGroq
from langchain.prompts.prompt import PromptTemplate

# Load environment variables
load_dotenv()



c:\Users\peter\Desktop\ds_ai\repo_folder\ds-rag-pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
warnings.filterwarnings("ignore")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

---
## Data Ingestion

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

def load_multiple_pdfs(folder_path):
    """
    Load text data from all PDF files in a folder.
    """
    documents = []
    pdf_folder = Path(folder_path)
    
    # Find all PDF files in the folder
    for pdf_file in pdf_folder.glob("*.pdf"):
        print(f"Loading {pdf_file.name}...")
        loader = PyPDFLoader(file_path=str(pdf_file))
        documents.extend(loader.load())
    
    return documents

# Load all PDFs from folder
medi_docs = load_multiple_pdfs(folder_path="../documents/DataBase")
print(f"Loaded {len(medi_docs)} pages total")
print(medi_docs[0].page_content)

Loading Adiponectin Role in Neurodegenerative Diseases.pdf...
Loading Correlation_of_Neurodegenerative_Diseases_with_Oxi.pdf...
Loading Diet, Nutrition and Chronic Degenerative Diseases.pdf...
Loading Educommunication in Nutrition and Neurodegenerative.pdf...
Loading Frailty, Cognitive Decline, Neurodegenerative.pdf...
Loading Ketogenic Diet An Effective Treatment Approach for Neurodegenerative.pdf...
Loading Mediterranean Diet and Neurodegenerative Diseases.pdf...
Loading Modulation of Gut Microbiota Through Dietary Intervention.pdf...
Loading Molecular Crossfires between Inflammasome Signalling and Dietary Small.pdf...
Loading Nutritional_Approaches_in_Neurodegenerative_Disord.pdf...
Loading The Ketogenic Diet and Alzheimer’s Disease.pdf...
Loading The Role of Diet and Dietary Patterns in Parkinson’s Disease.pdf...
Loading The Role of Diet in Parkinson’s Disease.pdf...
Loading The Role of Ketogenic Diet in the Treatment of Neurological Diseases.pdf...
Loading the_role_of_nutrition_in

---

## Document Chunking


In [19]:
# Import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Example chunking function
def split_documents(documents, chunk_size=200, chunk_overlap=50):
    """
    Splits documents into chunks of given size and overlap
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    
    # Just to add id for etch chunks to map it later 
    for i, chunk in enumerate(chunks):
         chunk.metadata.update({
        "id": f"chunk_{i}",
    })
    
    return chunks

In [20]:
# Execute your chunking function and display results here
medi_chunks = split_documents(medi_docs)


---

## Embedding and Storage


In [21]:
# Import libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
import numpy as np

# Example function for embeddings and storage

def create_embedding_vector_db(chunks, db_name):
    """
    This function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a VectorStore called FAISS, 
    which allows for efficient similarity search
    """
    # instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    # create the vector store 
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding,
        distance_strategy=DistanceStrategy.COSINE  # or DistanceStrategy.DOT or DistanceStrategy.L2 
        
    )
    # save VectorStore locally
    vectorstore.save_local(f"../vector_databases/vector_db_{db_name}")
    return vectorstore

In [22]:
# Generate embeddings and save them locally
all_embedding=create_embedding_vector_db(chunks=medi_chunks, db_name="medi")

---

## Retrieval from FAISS

In [23]:
# Implement retrieval logic from your FAISS database
def retrieve_from_vector_db(vector_db_path):
    """
    this function splits out a retriever object from a local VectorStore
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
        distance_strategy=DistanceStrategy.COSINE
    )
    retriever = react_vectorstore.as_retriever()
    return retriever ,react_vectorstore

# Load the retriever and index
react_retriever,react_vectorstore = retrieve_from_vector_db("../vector_databases/vector_db_medi")
type(react_retriever),type(react_vectorstore)

(langchain_core.vectorstores.base.VectorStoreRetriever,
 langchain_community.vectorstores.faiss.FAISS)

In [24]:
# Test your retrieval system with queries
query="""
what is import for diets of people with neurodegenerative diseases?
"""

In [25]:
react_retriever.get_relevant_documents(query,k=3)

[Document(id='e84c241c-8895-4c8b-83cb-d5506437672e', metadata={'producer': 'Adobe PDF Library 16.0', 'creator': 'Elsevier', 'creationdate': '2022-06-08T07:29:37+02:00', 'crossmarkdomains[2]': 'elsevier.com', 'crossmarkmajorversiondate': '2010-04-23', 'subject': 'The Journal of Notification, Health & Aging, 26 (2022) 606-614. doi:10.1007/s12603-022-1807-7', 'author': 'Haley Hersant', 'keywords': "Ketogenic diet; Alzheimer's disease; dementia; ketone bodies therapy", 'elsevierwebpdfspecifications': '6.5', 'crossmarkdomainexclusive': 'true', 'robots': 'noindex', 'moddate': '2023-11-29T14:23:38+05:30', 'doi': '10.1007/s12603-022-1807-7', 'crossmarkdomains[1]': 'sciencedirect.com', 'title': "The Ketogenic Diet and Alzheimer's Disease", 'trapped': '/False', 'source': '..\\documents\\DataBase\\The Ketogenic Diet and Alzheimer’s Disease.pdf', 'total_pages': 9, 'page': 0, 'page_label': '606', 'id': 'chunk_4707'}, page_content='specific diet that has been studied vis a vis neurodegenerative dise

---

## Connecting Retrieval with LLM

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Write a function to create retrieval and document processing chains
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain


react_retrieval_chain = connect_chains(react_retriever)

In [27]:
# Invoke your chain with a sample question
output = react_retrieval_chain.invoke(
    {"input": "what are neurodegenerative diseases?"}
)
type(output) , output.keys()

(dict, dict_keys(['input', 'context', 'answer']))

---

## Interactive Chat System


In [28]:
medi_retriever = retrieve_from_vector_db("../vector_databases/vector_db_medi")

medi_retrieval_chain = connect_chains(medi_retriever[0])

In [29]:
# Define your interactive chat querying function
def print_output(
    inquiry,
    retrieval_chain=react_retrieval_chain
):
    result = retrieval_chain.invoke({"input": inquiry})
    print(result['answer'].strip("\n"))

In [ ]:
# Run and test your interactive chat system
print_output("wie sollten sich menschen mit alzheimer ernähren?")

Basierend auf den bereitgestellten Kontextinformationen gibt es keine spezifischen Ernährungsempfehlungen für Menschen mit Alzheimer. Es gibt jedoch einige allgemeine Hinweise, die für eine gesunde Ernährung bei Alzheimer relevant sein könnten:

- Eine Studie von Morris et al. (2011) [1] erwähnt, dass eine Ernährung, die reich an Nährstoffen ist, wichtig für die Vorbeugung und Behandlung von Alzheimer ist.
- Eine andere Studie von Barczak (2022) [2] empfiehlt, dass Menschen mit Alzheimer eine ausgewogene Ernährung einnehmen sollten, die reich an Obst, Gemüse, Vollkornprodukten und gesunden Fetten ist.
- Eine Studie von J. Prev. Alzheimer’s Dis. (2021) [3] erwähnt, dass eine Ernährung, die reich an Antioxidantien ist, wie z.B. Vitamin C und E, wichtig für die Vorbeugung von Alzheimer ist.

Es ist jedoch wichtig zu beachten, dass diese Informationen sehr allgemein sind und dass eine individuelle Ernährungsberatung durch einen Arzt oder einen Ernährungsexperten erforderlich ist, um eine g

In [31]:
# Define your interactive chat querying function
def chat_with_rag(chain):
    """
    Interactive function to chat with the RAG system.
    """
    print("Welcome to the RAG Chat! Type 'exit' to quit.\n")
    while True:
        user_input = input("🧑 You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Exiting the chat. Goodbye!")
            break
        try:
            result = chain.invoke({"input": user_input})
            print(f"🤖 RAG Answer: {result['answer']}\n")
        except Exception as e:
            print(f" Error: {e}\n")


In [32]:
# Run your interactive chat
chat_with_rag(react_retrieval_chain)

Welcome to the RAG Chat! Type 'exit' to quit.

🤖 RAG Answer: Based on the provided context, I can offer some general nutritional advice for people with ALS. However, please note that this is not a substitute for personalized medical advice. It's essential to consult a healthcare professional or a registered dietitian for specific guidance.

From the context, we know that a diet rich in fruits and vegetables can confer protection against the development of ALS. Here are some general nutritional tips:

1. **Eat a balanced diet**: Focus on whole, unprocessed foods like fruits, vegetables, whole grains, lean proteins, and healthy fats.
2. **Increase antioxidant intake**: Antioxidants, such as vitamins C and E, can help protect against oxidative stress, which may contribute to ALS progression. Include antioxidant-rich foods like berries, leafy greens, and nuts in your diet.
3. **Stay hydrated**: Adequate hydration is essential for overall health, and it may help alleviate symptoms like musc